# Prediction

In [2]:
# packages
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

from xgboost import XGBClassifier

## carga de dados

In [3]:
# carregar os dados
alvo = np.genfromtxt('../data/heart_target.csv', delimiter=',', encoding='utf-8')
previsores = {
    "label_encoded": pd.read_csv('../data/heart_predictor_le.csv', sep=',', encoding='utf-8'),
    "one_hot_encoded": pd.read_csv('../data/heart_predictor_ohe.csv', sep=',', encoding='utf-8'),
    "escalonado": pd.read_csv('../data/heart_predictor_esc.csv', sep=',', encoding='utf-8')
}

### bases de dados: treino e teste

In [4]:
# bases de treino e teste
def separar_bases(previsores, alvo):
    data = {}
    for nome, df in previsores.items():
        x_train, x_test, y_train, y_test = train_test_split(df.values, alvo, test_size=0.3, random_state=83)
        data[f'x_train_{nome}'] = x_train
        data[f'x_test_{nome}'] = x_test
        data[f'y_train_{nome}'] = y_train
        data[f'y_test_{nome}'] = y_test
    return data

dados = separar_bases(previsores, alvo)

## modelos

In [5]:
# função para treinar avaliar um modelo

def treinar_e_avaliar_modelo(clf, nome_modelo, dados, previsores, alvo, cv_splits):
    """
    Treina, avalia o modelo para cada conjunto de previsores e realiza cross-validation.
    Retorna um DataFrame com as principais métricas (acurácia de treino, teste, validação cruzada, etc.).
    """
    resultados = [] 

    for nome, df in previsores.items():
        x_train = dados[f'x_train_{nome}']
        y_train = dados[f'y_train_{nome}']
        x_test = dados[f'x_test_{nome}']
        y_test = dados[f'y_test_{nome}']

        # treino
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)

        # metricas
        acuracia_teste = accuracy_score(y_test, y_pred)
        acuracia_treino = clf.score(x_train, y_train)
        matriz_confusao = confusion_matrix(y_test, y_pred)
        report = classification_report(y_test, y_pred, output_dict=True)

        # validação cruzada
        cv_scores = cross_val_score(clf, df, alvo, cv=StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=8))
        acuracia_cv = cv_scores.mean() * 100

        # Registro dos resultados
        resultados.append({
            'modelo': nome_modelo,
            'conjunto': nome,
            'acuracia_teste': acuracia_teste * 100,
            'acuracia_treino': acuracia_treino * 100,
            'acuracia_cv': acuracia_cv,
            'precision': report['weighted avg']['precision'] * 100,
            'recall': report['weighted avg']['recall'] * 100,
            'f1_score': report['weighted avg']['f1-score'] * 100,
            'matriz_confusao': matriz_confusao.tolist()
        })

    return pd.DataFrame(resultados)

## classificadores

### naive bayes

In [6]:
# naive bayes
naive = GaussianNB()
resultados_naive_bayes = treinar_e_avaliar_modelo(naive, "Naive Bayes", dados, previsores, alvo, 20)
resultados_naive_bayes

,modelo,conjunto,acuracia_teste,acuracia_treino,acuracia_cv,precision,recall,f1_score,matriz_confusao
0,Naive Bayes,label_encoded,86.956522,83.151326,84.178744,86.956522,86.956522,86.956522,"[[103, 18], [18, 137]]"
1,Naive Bayes,one_hot_encoded,86.956522,84.867395,85.164251,86.987942,86.956522,86.967630,"[[104, 17], [19, 136]]"
2,Naive Bayes,escalonado,86.956522,84.867395,85.164251,86.987942,86.956522,86.967630,"[[104, 17], [19, 136]]"


### support vector machines

In [7]:
# svm
svm = SVC(kernel='rbf', random_state=1, C=2)
resultados_svm = treinar_e_avaliar_modelo(svm, "SVM", dados, previsores, alvo, 20)
resultados_svm

,modelo,conjunto,acuracia_teste,acuracia_treino,acuracia_cv,precision,recall,f1_score,matriz_confusao
0,SVM,label_encoded,76.449275,68.798752,70.657005,76.408778,76.449275,76.254736,"[[82, 39], [26, 129]]"
1,SVM,one_hot_encoded,76.449275,68.486739,70.661836,76.446259,76.449275,76.214646,"[[81, 40], [25, 130]]"
2,SVM,escalonado,88.405797,91.107644,86.350242,88.579681,88.405797,88.319381,"[[99, 22], [10, 145]]"


### logistic regression

In [8]:
# regressao logistica
reg_log =  LogisticRegression(random_state=1, max_iter=2000, penalty="l2", tol=0.0001, C=1, solver="lbfgs")
resultados_regressao_logistica = treinar_e_avaliar_modelo(reg_log, "Logistic Regression", dados, previsores, alvo, 20)
resultados_regressao_logistica

,modelo,conjunto,acuracia_teste,acuracia_treino,acuracia_cv,precision,recall,f1_score,matriz_confusao
0,Logistic Regression,label_encoded,86.956522,84.087363,84.722222,86.956522,86.956522,86.914545,"[[100, 21], [15, 140]]"
1,Logistic Regression,one_hot_encoded,88.043478,85.647426,86.137681,88.349298,88.043478,87.924359,"[[97, 24], [9, 146]]"
2,Logistic Regression,escalonado,88.405797,85.803432,86.135266,88.777741,88.405797,88.279903,"[[97, 24], [8, 147]]"


### KNN

In [9]:
# knn
knn =  KNeighborsClassifier(n_neighbors=7, metric='minkowski', p=1)
resultados_knn = treinar_e_avaliar_modelo(knn, "KNN", dados, previsores, alvo, 20)
resultados_knn

,modelo,conjunto,acuracia_teste,acuracia_treino,acuracia_cv,precision,recall,f1_score,matriz_confusao
0,KNN,label_encoded,74.637681,78.939158,72.188406,74.570920,74.637681,74.406965,"[[79, 42], [28, 127]]"
1,KNN,one_hot_encoded,75.724638,80.499220,74.698068,75.668565,75.724638,75.524112,"[[81, 40], [27, 128]]"
2,KNN,escalonado,88.768116,88.143526,86.352657,88.909988,88.768116,88.693137,"[[100, 21], [10, 145]]"


### tree

In [10]:
# arvores de decisao
tree =  DecisionTreeClassifier(criterion='entropy', random_state = 0, max_depth=3)
resultados_tree = treinar_e_avaliar_modelo(tree, "Tree", dados, previsores, alvo, 20)
resultados_tree

,modelo,conjunto,acuracia_teste,acuracia_treino,acuracia_cv,precision,recall,f1_score,matriz_confusao
0,Tree,label_encoded,86.231884,85.335413,83.635266,86.256119,86.231884,86.169715,"[[98, 23], [15, 140]]"
1,Tree,one_hot_encoded,86.231884,85.335413,83.635266,86.256119,86.231884,86.169715,"[[98, 23], [15, 140]]"
2,Tree,escalonado,86.231884,85.335413,83.526570,86.256119,86.231884,86.169715,"[[98, 23], [15, 140]]"


### random forest

In [11]:
# random forest
random_for = RandomForestClassifier(n_estimators=150, criterion='entropy', random_state = 0, max_depth=4)
resultados_random_forest = treinar_e_avaliar_modelo(random_for, "Random Forest", dados, previsores, alvo, 20)
resultados_random_forest

,modelo,conjunto,acuracia_teste,acuracia_treino,acuracia_cv,precision,recall,f1_score,matriz_confusao
0,Random Forest,label_encoded,88.043478,87.675507,86.352657,88.173127,88.043478,87.963662,"[[99, 22], [11, 144]]"
1,Random Forest,one_hot_encoded,88.043478,86.583463,86.021739,88.173127,88.043478,87.963662,"[[99, 22], [11, 144]]"
2,Random Forest,escalonado,88.043478,86.583463,86.021739,88.173127,88.043478,87.963662,"[[99, 22], [11, 144]]"


### xgboost

In [12]:
# xgboost
xgboost = XGBClassifier(max_depth=2, learning_rate=0.05, n_estimators=250, objective='binary:logistic', random_state=3)
resultados_xgboost = treinar_e_avaliar_modelo(xgboost, "XGBoost", dados, previsores, alvo, 8)
resultados_xgboost

,modelo,conjunto,acuracia_teste,acuracia_treino,acuracia_cv,precision,recall,f1_score,matriz_confusao
0,XGBoost,label_encoded,89.130435,90.327613,87.561022,89.243196,89.130435,89.066014,"[[101, 20], [10, 145]]"
1,XGBoost,one_hot_encoded,88.768116,90.951638,87.234935,88.909988,88.768116,88.693137,"[[100, 21], [10, 145]]"
2,XGBoost,escalonado,88.768116,90.951638,87.234935,88.909988,88.768116,88.693137,"[[100, 21], [10, 145]]"


## resultados


In [13]:
# metricas de todos os modelos
resultados = pd.concat([resultados_knn, 
                        resultados_naive_bayes, 
                        resultados_random_forest, 
                        resultados_regressao_logistica, 
                        resultados_svm, 
                        resultados_tree, 
                        resultados_xgboost], 
                       ignore_index=True)

resultados['dif_acuracia_treino_teste'] = resultados['acuracia_treino'] - resultados['acuracia_teste']
resultados

,modelo,conjunto,acuracia_teste,acuracia_treino,acuracia_cv,precision,recall,f1_score,matriz_confusao,dif_acuracia_treino_teste
0,KNN,label_encoded,74.637681,78.939158,72.188406,74.570920,74.637681,74.406965,"[[79, 42], [28, 127]]",4.301476
1,KNN,one_hot_encoded,75.724638,80.499220,74.698068,75.668565,75.724638,75.524112,"[[81, 40], [27, 128]]",4.774582
2,KNN,escalonado,88.768116,88.143526,86.352657,88.909988,88.768116,88.693137,"[[100, 21], [10, 145]]",-0.624590
3,Naive Bayes,label_encoded,86.956522,83.151326,84.178744,86.956522,86.956522,86.956522,"[[103, 18], [18, 137]]",-3.805196
4,Naive Bayes,one_hot_encoded,86.956522,84.867395,85.164251,86.987942,86.956522,86.967630,"[[104, 17], [19, 136]]",-2.089127
5,Naive Bayes,escalonado,86.956522,84.867395,85.164251,86.987942,86.956522,86.967630,"[[104, 17], [19, 136]]",-2.089127
6,Random Forest,label_encoded,88.043478,87.675507,86.352657,88.173127,88.043478,87.963662,"[[99, 22], [11, 144]]",-0.367971
7,Random Forest,one_hot_encoded,88.043478,86.583463,86.021739,88.173127,88.043478,87.963662,"[[99, 22], [11, 144]]",-1.460015
8,Random Forest,escalonado,88.043478,86.583463,86.021739,88.173127,88.043478,87.963662,"[[99, 22], [11, 144]]",-1.460015
9,Logistic Regression,label_encoded,86.956522,84.087363,84.722222,86.956522,86.956522,86.914545,"[[100, 21], [15, 140]]",-2.869158


In [14]:
# top 3 acuracia em testes
melhores_acuracia_teste = resultados.nlargest(3, 'acuracia_teste')
print(melhores_acuracia_teste[['modelo', 'conjunto', 'acuracia_teste']])

     modelo         conjunto  acuracia_teste
18  XGBoost    label_encoded       89.130435
2       KNN       escalonado       88.768116
19  XGBoost  one_hot_encoded       88.768116


In [15]:
# top 3 validação cruzada
melhores_acuracia_cv = resultados.nlargest(3, 'acuracia_cv')
print(melhores_acuracia_cv[['modelo', 'conjunto', 'acuracia_cv']])

     modelo         conjunto  acuracia_cv
18  XGBoost    label_encoded    87.561022
19  XGBoost  one_hot_encoded    87.234935
20  XGBoost       escalonado    87.234935


In [16]:
# top 3 possivel overfiting
overfitting_potencial = resultados.nlargest(3, 'dif_acuracia_treino_teste')
print(overfitting_potencial[['modelo', 'conjunto', 'acuracia_teste', 'acuracia_treino', 'dif_acuracia_treino_teste']])

   modelo         conjunto  acuracia_teste  acuracia_treino  \
1     KNN  one_hot_encoded       75.724638        80.499220   
0     KNN    label_encoded       74.637681        78.939158   
14    SVM       escalonado       88.405797        91.107644   

    dif_acuracia_treino_teste  
1                    4.774582  
0                    4.301476  
14                   2.701847  


In [17]:
# top 3 f1_score
melhores_f1_score = resultados.nlargest(3, 'f1_score')
print(melhores_f1_score[['modelo', 'conjunto', 'f1_score']])

     modelo         conjunto   f1_score
18  XGBoost    label_encoded  89.066014
2       KNN       escalonado  88.693137
19  XGBoost  one_hot_encoded  88.693137


In [18]:
# top 3 precision
melhores_precision = resultados.nlargest(3, 'precision')
print(melhores_f1_score[['modelo', 'conjunto', 'precision']])

     modelo         conjunto  precision
18  XGBoost    label_encoded  89.243196
2       KNN       escalonado  88.909988
19  XGBoost  one_hot_encoded  88.909988


In [19]:
# top 3 recall
melhores_recall = resultados.nlargest(3, 'recall')
print(melhores_f1_score[['modelo', 'conjunto', 'recall']])

     modelo         conjunto     recall
18  XGBoost    label_encoded  89.130435
2       KNN       escalonado  88.768116
19  XGBoost  one_hot_encoded  88.768116


**Com base nos resultados o modelo XGBoost apresentou mais robustez sendo o melhor modelo geral**


In [20]:
# melhor modelo
resultados[(resultados['modelo'] == 'XGBoost') & (resultados['conjunto'] == 'label_encoded')]

,modelo,conjunto,acuracia_teste,acuracia_treino,acuracia_cv,precision,recall,f1_score,matriz_confusao,dif_acuracia_treino_teste
18,XGBoost,label_encoded,89.130435,90.327613,87.561022,89.243196,89.130435,89.066014,"[[101, 20], [10, 145]]",1.197178


In [21]:
# export
resultados.to_csv('../data/results.csv', sep=',', index=False)